<a href="https://colab.research.google.com/github/YuvalYurzd/AI_Firewall/blob/main/Roberta_finetuned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import necessary libraries
import pandas as pd
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from torch.utils.data import DataLoader, TensorDataset

# Load dataset
df = pd.read_csv(r'final_of_finals_v7.csv')
df = df.dropna(subset=['prompt'])
df = df[df['prompt'].apply(lambda x: isinstance(x, str))]
df = df.reset_index(drop=True)

# Split dataset into training and testing sets
train_texts, test_texts, train_labels, test_labels = train_test_split(df['prompt'].values, df['label'].values, test_size=0.2, random_state=42)

# Load the RoBERTa tokenizer and encode your texts
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True)
test_encodings = tokenizer(test_texts.tolist(), truncation=True, padding=True)

# Convert labels to tensors
train_labels = torch.tensor(train_labels)
test_labels = torch.tensor(test_labels)

# Create datasets and data loaders
train_dataset = TensorDataset(torch.tensor(train_encodings['input_ids']), torch.tensor(train_encodings['attention_mask']), train_labels)
test_dataset = TensorDataset(torch.tensor(test_encodings['input_ids']), torch.tensor(test_encodings['attention_mask']), test_labels)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Fine-tune the RoBERTa model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=4)
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)
print("Model training begins")

epochs = 3
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        batch = tuple(t.to(device) for t in batch)
        inputs = {'input_ids': batch[0],
                  'attention_mask': batch[1],
                  'labels': batch[2]}
        optimizer.zero_grad()
        outputs = model(**inputs)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{epochs}, Average Training Loss: {avg_train_loss:.4f}")

# Evaluate the fine-tuned model
model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for batch in test_loader:
        batch = tuple(t.to(device) for t in batch)
        inputs = {'input_ids': batch[0],
                  'attention_mask': batch[1]}
        outputs = model(**inputs)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1).cpu().numpy()
        labels = batch[2].cpu().numpy()
        predictions.extend(preds.tolist())
        true_labels.extend(labels.tolist())

accuracy = accuracy_score(true_labels, predictions)
print(f"Accuracy: {accuracy}")

report = classification_report(true_labels, predictions)
print(report)

# Save the model and tokenizer
output_dir = './fine_tuned_roberta'
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print(f"Model and tokenizer saved to {output_dir}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Model training begins
Epoch 1/3, Average Training Loss: 0.0643
Epoch 2/3, Average Training Loss: 0.0284
Epoch 3/3, Average Training Loss: 0.0250
Accuracy: 0.9892162019989479
              precision    recall  f1-score   support

           0       1.00      0.97      0.98      6775
           1       1.00      1.00      1.00      4257
           2       0.97      0.99      0.98      4557
           3       0.99      1.00      0.99      3421

    accuracy                           0.99     19010
   macro avg       0.99      0.99      0.99     19010
weighted avg       0.99      0.99      0.99     19010

Model and tokenizer saved to ./fine_tuned_roberta


In [3]:
model.save_pretrained("your_model_name.h5")


In [5]:
model.save_pretrained("fine_tuned_roberta_model")
tokenizer.save_pretrained("fine_tuned_roberta_tokenizer")

('fine_tuned_roberta_tokenizer/tokenizer_config.json',
 'fine_tuned_roberta_tokenizer/special_tokens_map.json',
 'fine_tuned_roberta_tokenizer/vocab.json',
 'fine_tuned_roberta_tokenizer/merges.txt',
 'fine_tuned_roberta_tokenizer/added_tokens.json')